In [1]:
import ROOT
import numpy as np
from matplotlib import pyplot as plt
import array
import os, sys
from multiprocessing import Pool
from matplotlib import rcParams
%jsroot on
rcParams['axes.titlepad'] = 20 

rate, efficiency = np.loadtxt("MVA-points.txt", unpack = True)

Welcome to JupyROOT 6.28/06


## GENERATE THE DATA

In [ ]:
# DATA GENERATION
simulationCPP = "LoopLineShape"
mvaScan = "\"true\""
ConfFile = "\"ToyConfiguration.txt\""
Nfiles = "1000"

def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

ROOT.gInterpreter.ProcessLine(".L LoopLineShape.cpp")

def generate(i):
    CosmicRate = str(rate[i])
    Efficiency = str(efficiency[i])
    folder = "mva_" + str(i)
    if not os.path.exists(folder):	# Check if the folder exist
        os.makedirs(folder)	# Create the folder
    code = simulationCPP + "(" + Nfiles + ",\"" + folder + "/\"," + mvaScan + "," + CosmicRate + "," + Efficiency + "," + ConfFile + ")"
    print("running: ", code)
    ROOT.LoopLineShape(int(Nfiles),folder + "/","true", rate[i], efficiency[i],"ToyConfiguration.txt")
    os.popen("cp ToyConfiguration.txt " + folder + "/"  + "ToyConfiguration.txt")	

# Paraller generation of the data
njob = 7
indexes = range(0,len(rate))
for i in range(0, int(len(rate)/njob)):
    #Create list of point to be generated
    start = i*njob
    stop = (i+1)*njob
    index = np.asarray(indexes[start:stop])
    print("Generating MVA points: ", index)
    if __name__ == "__main__":
        # Generate Files
        with Pool(processes = njob, maxtasksperchild = 1) as pool:
            pool.map(generate, index)
    # Create list of last points to be generated
    if (i == (int(len(rate)/njob) - 1) and (len(rate))%njob != 0):
        start = (i+1)*njob
        stop = start + len(rate)%njob
        index = np.asarray(indexes[start:stop])
        print("generating last points: " , index)
        # Generate last points
        if __name__ == "__main__":
            with Pool(processes = njob, maxtasksperchild = 1) as pool:
                pool.map(generate, index)
for i in range(0,len(rate)):
    folder = "mva_" + str(i)
    replace_line(folder + "/ToyConfiguration.txt", 17, "CosmicRate = %f\n" % (rate[i]))
    replace_line(folder + "/ToyConfiguration.txt", 18, "Efficiency = %f\n" % (efficiency[i]))

## ANALYZE THE DATA

In [2]:
# Constant Fraction
bias_cf = np.array([])
devStandard_cf = np.array([])
# forward 2017
bias_fw = np.array([])
devStandard_fw = np.array([])
# Reversed 2017
bias_rev = np.array([])
devStandard_rev = np.array([])
# Sum Neighbors 
bias_sum = np.array([])
devStandard_sum = np.array([])
# Threshold
bias_thr = np.array([])
devStandard_thr = np.array([])
# Classic Costant Fraction
bias_classic = np.array([])
devStandard_classic = np.array([])
# Hybrid method
bias_hybrid = np.array([])
devStandard_hybrid = np.array([])

# DATA ANALYSIS
# Execute analysis code
ROOT.gInterpreter.ProcessLine(".L ScanAnalysis.cpp")
folder = "mva_"
trial = 999
Nsum = 5

def task(i):
	values = ROOT.ScanAnalysis(folder + str(i) + "/", folder +  str(i) + "/ToyConfiguration.txt",0,trial,3,0.3, Nsum,rate[i])
	npResult = np.asarray(values)
	return npResult

if __name__ == "__main__":
	# create the process pool
	index = range(0,len(rate))
	with Pool(processes = 10, maxtasksperchild = 1) as pool:
		for results in pool.map(task, index):
			bias_thr = np.append(bias_thr, results[0])
			devStandard_thr = np.append(devStandard_thr, results[1])
			bias_fw = np.append(bias_fw, results[2])
			devStandard_fw = np.append(devStandard_fw, results[3])
			bias_rev = np.append(bias_rev, results[4])
			devStandard_rev = np.append(devStandard_rev, results[5])
			bias_cf = np.append(bias_cf, results[6])
			devStandard_cf = np.append(devStandard_cf, results[7])
			bias_sum = np.append(bias_sum, results[8])
			devStandard_sum = np.append(devStandard_sum, results[9]); bias_classic = np.append(bias_classic,results[10]); devStandard_classic = np.append(devStandard_classic, results[11]); bias_hybrid = np.append(bias_hybrid,results[12]); devStandard_hybrid = np.append(devStandard_hybrid,results[13])

AttributeError: Failed to get attribute ScanAnalysis from ROOT

In file included from input_line_56:1:
/home/adriano/Documents/ALPHA/ALPHA-2/LineShape/ScanAnalysis.cpp:135:117: error: expected ';' after expression
                diff_hybrid.push_back(onset2 - onset1 - (Params.x_da_start + lineShiftda[0] - Params.x_cb_start - lineShiftcb[0]))
                                                                                                                                  ^
                                                                                                                                  ;


# PLOT THE DATA

In [ ]:
# define a cost function, function of bias and devStandard
def cost(bias,dev):
    # funzione costo : (deviazione standard + bias)**2/deviazione standard
    return np.sqrt((dev + np.abs(bias))**2)

cost_cf = cost(bias_cf,devStandard_cf)	
cost_fw = cost(bias_fw,devStandard_fw)
cost_rev = cost(bias_rev,devStandard_rev)
cost_sum = cost(bias_sum,devStandard_sum)
cost_thr = cost(bias_thr,devStandard_thr)
cost_classic = cost(bias_classic,devStandard_classic)
cost_hybrid = cost(bias_hybrid,devStandard_hybrid)

plt.figure(0, figsize = (12,12))
plt.title("MVA curve")
plt.xscale('log')
plt.grid()
plt.errorbar(rate, efficiency, linestyle = '', marker = '.', color = 'red')

plt.figure(1, figsize = (12,12))
plt.title("MVA scan " + r"$N_{trial}$ = " + str(trial + 1) , fontsize = 20)
plt.xlabel("rate [events/second]", fontsize = 15)
plt.ylabel("cost [kHz]", fontsize = 15)
plt.xscale("log")
plt.grid()
plt.plot(0.051, 0.5, color = "black", linestyle = '', marker=r'$\downarrow$', markersize=15, label = "PassCut")
plt.errorbar(rate,cost_cf, linestyle = '',marker = "v", color = "red", label = "constant fraction 30%")
plt.errorbar(rate, cost_classic, linestyle = '', marker = "v", color = "grey", label = "costant fraction 30% (without cosmic)")
plt.errorbar(rate,cost_fw, linestyle = '',marker = ".", color = "blue", label = "forward", markersize = 8)
plt.errorbar(rate,cost_rev, linestyle = '',marker = "D", color = "green", label = "reversed")
plt.errorbar(rate,cost_sum, linestyle = '',marker = "s", color = "black", label = "Neighbors sum (N = %d)" % Nsum)
plt.errorbar(rate,cost_thr, linestyle = '',marker = "D", color = "orange", label = "over Threshold (> 3)")
plt.errorbar(rate,cost_hybrid, linestyle = '',marker = "hexagon1", color = "turquoise", label = "hybrid (fraction 30%)")
plt.legend(fontsize = 10)
plt.savefig("Plot/mvaScan/cost.jpg", format = "jpg")
plt.savefig("Plot/mvaScan/cost.pdf", format = "pdf")

plt.figure(2, figsize = (12,12))
plt.title("MVA scan " + r"$N_{trial}$ = " + str(trial + 1) , fontsize = 20)
plt.xlabel("rate [events/second]", fontsize = 15)
plt.ylabel(r"$\sigma$ [kHz]", fontsize = 15)
plt.xscale("log")
plt.grid()
plt.plot(0.051, 0.5, color = "black", linestyle = '', marker=r'$\downarrow$', markersize=15, label = "PassCut")
plt.errorbar(rate,devStandard_cf, linestyle = '',marker = "v", color = "red", label = "constant fraction 30%")
plt.errorbar(rate, devStandard_classic, linestyle = '', marker = "v", color = "grey", label = "costant fraction 30% (without cosmic)")
plt.errorbar(rate,devStandard_fw, linestyle = '',marker = ".", color = "blue", label = "forward", markersize = 8)
plt.errorbar(rate, devStandard_rev, linestyle = '',marker = "D", color = "green", label = "reversed")
plt.errorbar(rate,devStandard_sum, linestyle = '',marker = "s", color = "black", label = "Neighbors sum (N = %d)" % Nsum)
plt.errorbar(rate, devStandard_thr, linestyle = '',marker = "D", color = "orange", label = "over Threshold (> 3)")
plt.errorbar(rate,devStandard_hybrid, linestyle = '',marker = "hexagon1", color = "turquoise", label = "hybrid (fraction 30%)")
plt.legend(fontsize = 10)
plt.savefig("Plot/mvaScan/sigmaVSrate.jpg", format = "jpg")
plt.savefig("Plot/mvaScan/sigmaVSrate.pdf", format = "pdf")

plt.figure(3, figsize = (12,12))
plt.title("Bias " + r"$N_{trial}$ = " + str(trial + 1) , fontsize = 20)
plt.xlabel("rate [events/second]", fontsize = 15)
plt.ylabel("Bias", fontsize = 15)
plt.xscale("log")
plt.grid()
plt.errorbar(rate,np.abs(bias_cf), linestyle = '',marker = "v", color = "red", label = "constant fraction 30%")
plt.errorbar(rate,np.abs(bias_fw), linestyle = '',marker = ".", color = "blue", label = "forward", markersize = 8)
plt.errorbar(rate,np.abs(bias_rev), linestyle = '',marker = "D", color = "green", label = "reversed")
plt.errorbar(rate,np.abs(bias_sum), linestyle = '',marker = "s", color = "black", label = "Neighbors sum (N = %d)" % Nsum)
plt.errorbar(rate,np.abs(bias_thr), linestyle = '',marker = "D", color = "orange", label = "over Threshold (> 3)")
plt.legend(fontsize = 10)
plt.savefig("Plot/mvaScan/bias.jpg", format = "jpg")
plt.savefig("Plot/mvaScan/bias.pdf", format = "pdf")

plt.figure(4, figsize = (12,12))
plt.title(r"$\sigma$ vs Bias " + r"$N_{trial}$ = " + str(trial + 1) , fontsize = 20)
plt.xlabel(r"$\sigma$ [kHz]", fontsize = 15)
plt.ylabel("Bias [kHz]", fontsize = 15)
plt.grid()
plt.xlim(2.5,20)
plt.ylim(-1,4)
plt.errorbar(devStandard_cf,np.abs(bias_cf), linestyle = '',marker = "v", color = "red", label = "constant fraction 30%")
plt.errorbar(devStandard_fw,np.abs(bias_fw), linestyle = '',marker = ".", color = "blue", label = "forward", markersize = 8)
plt.errorbar(devStandard_rev,np.abs(bias_rev), linestyle = '',marker = "D", color = "green", label = "reversed")
plt.errorbar(devStandard_sum,np.abs(bias_sum), linestyle = '',marker = "s", color = "black", label = "Neighbors sum (N = %d)" % Nsum)
plt.errorbar(devStandard_thr,np.abs(bias_thr), linestyle = '',marker = "D", color = "orange", label = "over Threshold (> 3)")
plt.legend(fontsize = 10)
plt.savefig("Plot/mvaScan/biasVSsigma.jpg", format = "jpg")
plt.savefig("Plot/mvaScan/biasVSsigma.pdf", format = "pdf")
plt.show()
